# Módulo 2: HTML: Requests y BeautifulSoup
## Parsing Pagina12

<img src='https://www.pagina12.com.ar/assets/media/logos/logo_pagina_12_n.svg?v=1.0.178' width=300></img>
En este módulo veremos cómo utilizar las bibliotecas `requests` y `bs4` para programar scrapers de sitios HTML. Nos propondremos armar un scraper de noticias del diario <a href='www.pagina12.com.ar'>Página 12</a>.

Supongamos que queremos leer el diario por internet. Lo primero que hacemos es abrir el navegador, escribir la URL del diario y apretar Enter para que aparezca la página del diario. Lo que ocurre en el momento en el que apretamos Enter es lo siguiente:
1. El navegador envía una solicitud a la URL pidiéndole información.
2. El servidor recibe la petición y procesa la respuesta.
3. El servidor envía la respuesta a la IP de la cual recibió la solicitud.
4. Nuestro navegador recibe la respuesta y la muestra **formateada** en pantalla.

Para hacer un scraper debemos hacer un programa que replique este flujo de forma automática para luego extraer la información deseada de la respuesta. Utilizaremos `requests` para realizar peticiones y recibir las respuestas y `bs4` para *parsear* la respuesta y extraer la información.<br>
Te dejo unos links que tal vez te sean de utilidad:
- [Códigos de status HTTP](https://developer.mozilla.org/es/docs/Web/HTTP/Status)
- [Documentación de requests](https://requests.kennethreitz.org/en/master/)
- [Documentación de bs4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [1]:
import requests

In [2]:
url = 'https://www.pagina12.com.ar/'

In [3]:
p12 = requests.get(url)

In [4]:
p12.status_code

200

In [5]:
p12.content

b'<!DOCTYPE html><html class="no-js"><head><meta charset="utf-8"><title>P\xc3\xa1gina12 | La otra mirada</title><meta name="google-site-verification" content="x6zSdT0DBcKDmridH4LpEVrCmxcOunR2dgBQVmuL6fg"><script type="application/ld+json">{"@context": "http://schema.org","@type": "Organization","name": "P\xc3\xa1gina12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}</script><meta property="description" name="description" content="Notas focalizadas en el quehacer pol\xc3\xadtico de la Argentina."><meta property="fb:pages" name="fb:pages" content="1541638399393436"><meta property="og:locale" name="og:locale" content="es_AR"><meta property="og:title" name="og:title" content="P\xc3\xa1gina12 

Muchas veces la respuesta a la solicitud puede ser algo que no sea un texto: una imagen, un archivo de audio, un video, etc.

In [6]:
p12.text

'<!DOCTYPE html><html class="no-js"><head><meta charset="utf-8"><title>Página12 | La otra mirada</title><meta name="google-site-verification" content="x6zSdT0DBcKDmridH4LpEVrCmxcOunR2dgBQVmuL6fg"><script type="application/ld+json">{"@context": "http://schema.org","@type": "Organization","name": "Página12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}</script><meta property="description" name="description" content="Notas focalizadas en el quehacer político de la Argentina."><meta property="fb:pages" name="fb:pages" content="1541638399393436"><meta property="og:locale" name="og:locale" content="es_AR"><meta property="og:title" name="og:title" content="Página12 | La otra mirada"><meta prope

Analicemos otros elementos de la respuesta

In [7]:
p12.headers

{'Date': 'Thu, 03 Dec 2020 02:45:51 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=d8ff4117dfe8142841b8b7626ab17a2571606963551; expires=Sat, 02-Jan-21 02:45:51 GMT; path=/; domain=.pagina12.com.ar; HttpOnly; SameSite=Lax', 'Vary': 'Accept-Encoding', 'X-DNS-Prefetch-Control': 'off', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'X-Download-Options': 'noopen', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'X-ETag': 'W/"676c0-9tA+cmEM2WilhW98BLG6rNgZXic"', 'X-Backend': 'frontend', 'X-Backend-TTL': '180.000', 'X-Type': 'Dynamic URI', 'Age': '3', 'grace': '86400.000 none', 'ttl': '116.862', 'X-Instance': 'cache-front-prod-varnish-7f9f4bb47d-s4k56', 'x-restarts': '0', 'X-Cache': 'HIT (10)', 'CF-Cache-Status': 'DYNAMIC', 'cf-request-id': '06c816f46b0000b2c082b6b000000001', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/be

In [8]:
p12.request.headers

{'User-Agent': 'python-requests/2.25.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

El contenido de la request que acabamos de hacer está avisando que estamos utilizando la biblioteca requests para python y que no es un navegador convencional. Se puede modificar

In [9]:
p12.cookies

<RequestsCookieJar[Cookie(version=0, name='__cfduid', value='d8ff4117dfe8142841b8b7626ab17a2571606963551', port=None, port_specified=False, domain='.pagina12.com.ar', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=1609555551, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None, 'SameSite': 'Lax'}, rfc2109=False)]>

In [10]:
from bs4 import BeautifulSoup

In [11]:
s = BeautifulSoup(p12.text, 'lxml')

In [12]:
type(s)

bs4.BeautifulSoup

In [13]:
print(s.prettify())

<!DOCTYPE html>
<html class="no-js">
 <head>
  <meta charset="utf-8"/>
  <title>
   Página12 | La otra mirada
  </title>
  <meta content="x6zSdT0DBcKDmridH4LpEVrCmxcOunR2dgBQVmuL6fg" name="google-site-verification"/>
  <script type="application/ld+json">
   {"@context": "http://schema.org","@type": "Organization","name": "Página12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}
  </script>
  <meta content="Notas focalizadas en el quehacer político de la Argentina." name="description" property="description"/>
  <meta content="1541638399393436" name="fb:pages" property="fb:pages"/>
  <meta content="es_AR" name="og:locale" property="og:locale"/>
  <meta content="Página12 | La otra mirada" na

Primer ejercicio: obtener un listado de links a las distintas secciones del diario.<br>
Usar el inspector de elementos para ver dónde se encuentra la información.

In [14]:
secciones = s.find('ul', attrs={'class':'hot-sections'}).find_all('li')
secciones

AttributeError: 'NoneType' object has no attribute 'find_all'

In [15]:
[seccion.text for seccion in secciones]

NameError: name 'secciones' is not defined

In [16]:
seccion = secciones[0]

NameError: name 'secciones' is not defined

In [17]:
seccion.a.get('href')

NameError: name 'seccion' is not defined

Estamos interesados en los links, no en el texto

In [18]:
links_secciones = [seccion.a.get('href') for seccion in secciones]
links_secciones

NameError: name 'secciones' is not defined

Carguemos la página de una sección para ver cómo se compone

In [19]:
sec = requests.get(links_secciones[0])

NameError: name 'links_secciones' is not defined

In [20]:
sec

NameError: name 'sec' is not defined

In [21]:
sec.request.url

NameError: name 'sec' is not defined

In [22]:
soup_seccion = BeautifulSoup(sec.text, 'lxml')

NameError: name 'sec' is not defined

In [23]:
print(soup_seccion.prettify())

NameError: name 'soup_seccion' is not defined

La página se divide en un artículo promocionado y una lista `<ul>` con el resto de los artículos

In [24]:
featured_article = soup_seccion.find('div', attrs={'class':'featured-article__container'})
featured_article

NameError: name 'soup_seccion' is not defined

In [25]:
featured_article.a.get('href')

NameError: name 'featured_article' is not defined

In [26]:
article_list = soup_seccion.find('ul', attrs={'class':'article-list'})

NameError: name 'soup_seccion' is not defined

In [ ]:
def obtener_notas(soup):
    '''
    Función que recibe un objeto de BeautifulSoup de una página de una sección
    y devuelve una lista de URLs a las notas de esa sección
    '''
    lista_notas = []
    
    # Obtengo el artículo promocionado
    featured_article = soup.find('div', attrs={'class':'featured-article__container'})
    if featured_article:
        lista_notas.append(featured_article.a.get('href'))
    
    # Obtengo el listado de artículos
    article_list = soup.find('ul', attrs={'class':'article-list'})
    for article in article_list.find_all('li'):
        if article.a:
            lista_notas.append(article.a.get('href'))
    
    return lista_notas

Probemos la función

In [ ]:
lista_notas = obtener_notas(soup_seccion)
lista_notas

## Clase 6
En esta clase te voy a hablar un poco del manejo de errores. Para eso vamos a tomar como ejemplo uno de los links que obtuvimos con la función que tenías que armar en la clase anterior.

Código de error != 200

In [ ]:
r = requests.get(lista_notas[0])
if r.status_code == 200:
    # Procesamos la respuesta
    print('procesamos..')
else:
     # Informar el error
    print('informamos...')

In [ ]:
url_nota = lista_notas[0]
print(url_nota)

Supongamos que el link a la nota está mal cargado, o que sacaron la nota del sitio, o que directamente no está funcionando la web de página 12.

In [ ]:
url_mala = url_nota.replace('2','3')
print(url_mala)

Esto lo hacemos sólo para simular una URL mal cargada o un sevidor caído

In [ ]:
r = requests.get(url_mala)
if r.status_code == 200:
    # Procesamos la respuesta
    print('procesamos..')
else:
     # Informar el error
    print('informamos status code != 200')

Obtuvimos un error que interrumpió la ejecución del código. No llegamos a imprimir el status code. Muchas veces estos errores son inevitables y no dependen de nosotros. Lo que sí depende de nosotros es cómo procesarlos y escribir un código que sea robusto y resistente a los errores.

In [ ]:
try:
    nota = requests.get(url_mala)
except:
    print('Error en la request!\n')
    
print('El resto del programa continúa...')

Las buenas prácticas de programación incluyen el manejo de errores para darle robustez al código

In [ ]:
try:
    nota = requests.get(url_mala)
except Exception as e:
    print('Error en la request:')
    print(e)
    print('\n')
    
print('El resto del programa continúa...')

Lo mismo ocurre cuando encadenamos búsquedas. Retomemos esta línea

In [ ]:
featured_article.a.get('href')

Si no existe el tag "a", obtendremos un error que dice que un objeto None no tiene ningún método .get('href')

In [ ]:
try:
    featured_article.a.get('href')
except:
    pass